## Dataset Content

<b>Context</b>

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. Predictive data analytics methods are easy to apply with this dataset.

<b>Attribute information</b>

- Invoice id: Computer generated sales slip invoice identification number
- Branch: Branch of supercenter (3 branches are available identified by A, B and C).
- City: Location of supercenters
- Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
- Gender: Gender type of customer
- Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
- Unit price: Price of each product in USD
- Quantity: Number of products purchased by customer
- Tax: 5% tax fee for customer buying
- Total: Total price including tax
- Date: Date of purchase (Record available from January 2019 to March 2019)
- Time: Purchase time (10am to 9pm)
- Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
- COGS: Cost of goods sold
- Gross margin percentage: Gross margin percentage
- Gross income: Gross income
- Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

### Ideas
- Is the customer behavior different depending on the branches?
- How is the time/workdays or weekend impacting the customer behavior?
- Columns to keep:

       ['Branch', 'Product line', 'Unit price', 'Quantity', 'Total', 'Date', 'Time', 'Rating']

## First look

In [1]:
# Importing librairies
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

In [2]:
# Importing file
retail_original = pd.read_csv('./dataset/supermarket_sales - Sheet1.csv')
retail_original.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [3]:
# Shape
retail_original.shape

(1000, 17)

In [4]:
# Type
retail_original.dtypes

Invoice ID                  object
Branch                      object
City                        object
Customer type               object
Gender                      object
Product line                object
Unit price                 float64
Quantity                     int64
Tax 5%                     float64
Total                      float64
Date                        object
Time                        object
Payment                     object
cogs                       float64
gross margin percentage    float64
gross income               float64
Rating                     float64
dtype: object

In [5]:
# Looking for nulls
retail_original.isnull().sum()

Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Total                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
dtype: int64

## Cleaning

In [6]:
# New DF with only used columns
retail = retail_original[['Branch', 'Product line', 'Unit price', 'Quantity', 'Total', 'Date', 'Time', 'Rating']]
retail.head()

,Branch,Product line,Unit price,Quantity,Total,Date,Time,Rating
0,A,Health and beauty,74.69,7,548.9715,1/5/2019,13:08,9.1
1,C,Electronic accessories,15.28,5,80.2200,3/8/2019,10:29,9.6
2,A,Home and lifestyle,46.33,7,340.5255,3/3/2019,13:23,7.4
3,A,Health and beauty,58.22,8,489.0480,1/27/2019,20:33,8.4
4,A,Sports and travel,86.31,7,634.3785,2/8/2019,10:37,5.3


In [7]:
# Joining Date and Time
retail["Date"] = retail["Date"] + ' ' + retail["Time"]

# Dropping Time column
retail = retail.drop(['Time'], axis = 1)

# Converting Datetime object to datetime
retail['Date'] = pd.to_datetime(retail['Date'])

# Get hour
retail['Hour'] = retail['Date'].dt.hour

# Get weekday: Monday = 0 - Sunday = 6
retail['Weekday'] = retail['Date'].dt.dayofweek

# Getting YearMonth
retail['YearMonth'] = pd.to_datetime(retail['Date']).dt.to_period('M')

/Users/marie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
retail.dtypes

Branch                  object
Product line            object
Unit price             float64
Quantity                 int64
Total                  float64
Date            datetime64[ns]
Rating                 float64
Hour                     int64
Weekday                  int64
YearMonth            period[M]
dtype: object

In [9]:
retail.head()

,Branch,Product line,Unit price,Quantity,Total,Date,Rating,Hour,Weekday,YearMonth
0,A,Health and beauty,74.69,7,548.9715,2019-01-05 13:08:00,9.1,13,5,2019-01
1,C,Electronic accessories,15.28,5,80.2200,2019-03-08 10:29:00,9.6,10,4,2019-03
2,A,Home and lifestyle,46.33,7,340.5255,2019-03-03 13:23:00,7.4,13,6,2019-03
3,A,Health and beauty,58.22,8,489.0480,2019-01-27 20:33:00,8.4,20,6,2019-01
4,A,Sports and travel,86.31,7,634.3785,2019-02-08 10:37:00,5.3,10,4,2019-02


In [10]:
# Reordering columns
retail = retail[['Branch', 'YearMonth', 'Weekday', 'Hour', 'Product line', 'Unit price', 'Quantity', 'Total', 'Rating']]

In [11]:
retail.head()

,Branch,YearMonth,Weekday,Hour,Product line,Unit price,Quantity,Total,Rating
0,A,2019-01,5,13,Health and beauty,74.69,7,548.9715,9.1
1,C,2019-03,4,10,Electronic accessories,15.28,5,80.2200,9.6
2,A,2019-03,6,13,Home and lifestyle,46.33,7,340.5255,7.4
3,A,2019-01,6,20,Health and beauty,58.22,8,489.0480,8.4
4,A,2019-02,4,10,Sports and travel,86.31,7,634.3785,5.3


In [12]:
# Saving retail clean to csv
retail.to_csv('./dataset/retail.csv')